Converts all context jsons into one single graph json

Example graph input:

```
{
  "nodes": [
    {"id": "Myriel", "group": 1},
    {"id": "Napoleon", "group": 1},
  ],
"links": [
    {"source": "Napoleon", "target": "Myriel", "value": 1},
    {"source": "Mlle.Baptistine", "target": "Myriel", "value": 8},
],
}
```

In [16]:
import json
import glob
import re

json_path = './'

In [20]:
files = glob.glob(json_path + '*_*_*.json')

In [34]:
course_set = set()
concept_set = set()
lecture_set = set()

nodes = []
links = []


for file in files:
    lec2course = {}
    lec2concept = {}

    # add course node
    course_name = file.split('_')[1]
    if course_name not in course_set:
        course_set.add(course_name)
        course_node = {
            'id': course_name, 
            'group': 0,
        }
        nodes.append(course_node)


    with open(file, 'r') as f:
        context = json.load(f)
        f.close()

    # add concept node
    # print(context)
    concept_name = context['text']
    if concept_name not in concept_set:
        concept_set.add(concept_name)
        concept_node = {
            'id': concept_name, 
            'group': 2,
        }
        nodes.append(concept_node)


    for d in context['contexts']:
        # add lecture node
        lecture_name = d['course'] + '_' + str(d['lecture_num'])
        if lecture_name not in lecture_set:
            lecture_set.add(lecture_name)
            lecture_node = {
                'id': lecture_name, 
                'group': 1,
                'context': d['context'], 
                'label': d['label'], 
                'lecture': d['lecture'], 
                'transcription': d['transcription'], 
            }
            nodes.append(lecture_node)

        # add links
        lec2course[lecture_name] = course_name
        lec2concept[lecture_name] = concept_name


    for lecture_name in lec2course:
        links.append(
            {
                "source": lecture_name, 
                "target": lec2course[lecture_name], 
                "value": 1
            }
        )
    for lecture_name in lec2concept:
        links.append(
            {
                "source": lecture_name, 
                "target": lec2concept[lecture_name], 
                "value": 1
            }
        )



In [36]:


with open('course_concepts.json', 'w') as f:
    json.dump({
        'nodes': nodes,
        'links': links,
    }, f, ensure_ascii=False, indent=4)